# Aula 3 - Exercícios Extra - Pandas, Fontes de dados e Agregação
### Consulta a internet (Google, Stackoverflow e etc) liberada!!

In [1]:
# Importe as bibliotecas pandas, numpy, glob e sqlalchemy
import pandas as pd
import numpy as np


### 3.9 Construa uma função para concatenar (verticalmente) o conteúdo de vários arquivos CSV do dataset de morbidade. Esses arquivos contêm dados com colunas compatíveis.
#### Extraia a data do cabeçalho do arquivo. Divida a coluna 'Município' em duas outras contendo o código ibge de 6 caracteres e o nome do município.



In [2]:
# Dica 1: Construa uma lista de arquivos presentes num diretório usando a função glob da biblioteca glob
# Dica 2: Num loop, leia os arquivo duas vezes. Na primeira vez, pra carregar somente a parte desejada do cabeçalho,
#         onde se encontra o mês/ano do respectivo arquivo, utilize o parâmetro nrows. Preste atenção ao separador 
#         dos arquivos CSV, é um ponto e vírgula.
# Dica 3: O método split pode dividir uma string na posição de um caracter especificado.
# Dica 4: Se quiser converter o nome do mês para o seu número, você precisa ajustar a configuração de locale
#         do python para 'pt_BR'. Use o código a seguir para fazer esta operação.
#            import locale
#            locale.setlocale(locale.LC_ALL, 'pt_BR')
# Dica 5: O código strptime(nome_mes,'%b').tm_mon converte nome do mês no seu respectivo número de 1 a 12
# Dica 6: Para criar uma data a partir dos seus atributos use um código semelhante ao exemplo a seguir
#            import datetime
#            datetime.date(year=2020, month=1, day=1)
# Dica 7: Para construir uma nova coluna aplicando uma regra para cada célula use a função apply e construa uma
#         função lambda.
# Dica 8: O método join de uma string consegue unir, com um caracter separador, o conteúdo de uma lista de 
#         elementos numa string. Exemplo: ' '.join(lista_exemplo), produz uma string concatenando com um espaçao
#         os elementos da lista_exemplo. Pode ser útil para isolar o nome do município.
# Dica 9: Adicione todos os dataframes construídos dentro do loop a uma lista e depois use o método concat para 
#         concatenar os dataframes verticalmente. Use o parâmetro ignore_index=True para ignorar o índice dos 
#         dataframes.

import glob
import re
import locale
import datetime
from time import strptime

def converte_mes(nome_abreviado):
    """
    Como o locale não estava funcionando no servidor e as abreviaçoes dos nomes dos meses nos arquivos
    estão em português, criei essa função para converter a abreviaçao do mês em português em seu 
    número correspondente
    """
    try:
        dict_meses = {'JAN':1, 'FEV':2, 'MAR':3, 'ABR':4, 'MAI':5, 'JUN':6, \
                      'JUL':7, 'AGO':8, 'SET':9, 'OUT':10, 'NOV':11, 'DEZ':12}
        return dict_meses[nome_abreviado.upper()]
        
    except ValueError as e:
        
        raise NotImplementedeError(e)

        
def concat_morbidade(files_path):
    """Retorne um dataframe com a concatenação vertical dos dados dos arquivos do diretorio dir_path
    """
    # YOUR CODE HERE
    try:
        locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
        lista_dates = []
        lista_dfs = []
        lista_arqs = glob.glob(files_path)
        
        for arq in lista_arqs:
            df_dtArq = pd.read_csv(arq, nrows=2, encoding='latin_1')
            periodo = re.split('\s+', df_dtArq.loc[1].to_string())[-1]
            #n_mes = converte_mes( re.split('\/', periodo.split(':')[1])[0] )
            n_mes = strptime(re.split('\/', periodo.split(':')[1])[0] ,'%b').tm_mon          
            date_arquivo = datetime.date(year=int(re.split('\/', periodo.split(':')[1])[1]), month=n_mes, day=1)
            lista_dates.append(date_arquivo)
            
            df_contArq = pd.read_csv(arq, sep=';', skiprows=3, skipfooter=6, encoding='latin_1', engine='python')
            df_contArq['cod_ibge6'] = df_contArq['Município'].apply( lambda x: x.split(' ')[0] )
            df_contArq['municipio'] = df_contArq['Município'].apply( lambda x: ' '.join( x.split(' ')[1:] ) )
            lista_dfs.append(df_contArq)
            
        return pd.concat(lista_dfs, keys=lista_dates, ignore_index=True)
        
    except ValueError as e:
        
        raise NotImplementedError(e)


In [4]:
## Faça seus testes aqui
files_path = '../../curso_ciencia_dados2020/data/originais/morbidade/2020/*.csv'

morbidade = concat_morbidade(files_path)
test_data = set(['10899,59', '16793,57', '6585,96', '7446,97', '19054,46', '11462,34'])

slice_dict = set(morbidade.loc[morbidade['cod_ibge6']=='522230', 'Valor_total'].to_dict().values())

slice_dict

{'10899,59', '11462,34', '16793,57', '19054,46', '6585,96', '7446,97'}

In [5]:
# Validação para nota
files_path = '../../curso_ciencia_dados2020/data/originais/morbidade/2020/*.csv'
test_data = set(['10899,59', '16793,57', '6585,96', '7446,97', '19054,46', '11462,34'])
morbidade = concat_morbidade(files_path)
slice_dict = set(morbidade.loc[morbidade['cod_ibge6']=='522230', 'Valor_total'].to_dict().values())

assert slice_dict == test_data

### 3.10 Escreva uma função para gerar uma permutação (aleatória) de uma lista e colocar o resultado num dataframe com os elementos da lista agrupados em grupos de tamanho N (Ex.: N=4). 
#### A coluna de índices deve conter o nome dos grupos. Os nomes dos grupos devem seguir a seguinte regra: Grupo 1, Grupo 2, Grupo 3,..., Grupo K.
#### Veja como ficaria o resultado para o exemplo a seguir com uma lista de 9 elementos e tamanho N=4.

|         | 0                  |
|:--------|:-------------------|
| Grupo 0 | Julie Lewis        |
| Grupo 0 | Mary Phinney       |
| Grupo 0 | Gilbert Ravenell   |
| Grupo 0 | Jillian Brennan    |
| Grupo 1 | Michelle Humphreys |
| Grupo 1 | Kimberly Ransome   |
| Grupo 1 | Lisa Watson        |
| Grupo 1 | Jennifer Daniels   |
| Grupo 2 | Olga Crocker       |

In [6]:
# Dica: A função len calcula o tamanho de uma lista.

def create_groups(names_list, n):
    """Crie um dataframe com os nomes da lista names_list agrupados em grupos de tamanho n.
    """
    # YOUR CODE HERE
    try:
        
        lst_index = []
        j = 0
        aux = 0
        for i in range( len(names_list) ):
            if aux < n:
                j = j
                aux += 1                
            else:
                j += 1
                aux = 1
            
            lst_index.append('Grupo {0}'.format(j))

        df_grupo = pd.DataFrame( [name for name in names_list], \
                                columns=['Nomes'], \
                                index=lst_index)
        return df_grupo

    except ValueError as e:
        raise NotImplementedError(e)


In [7]:
## Faça seus testes aqui
import names
N = 9
group_length = 4
test_data = [names.get_full_name() for n in range(N)]

teste = create_groups(test_data, group_length)
teste.index.to_list()


['Grupo 0',
 'Grupo 0',
 'Grupo 0',
 'Grupo 0',
 'Grupo 1',
 'Grupo 1',
 'Grupo 1',
 'Grupo 1',
 'Grupo 2']

In [8]:
# Validação para nota
import names
N = 9
group_length = 4
test_data = [names.get_full_name() for n in range(N)]
result = ['Grupo 0', 'Grupo 0', 'Grupo 0', 'Grupo 0', 'Grupo 1', 'Grupo 1', 'Grupo 1', 'Grupo 1', 'Grupo 2']
df = create_groups(test_data, group_length)

assert df.index.to_list() == result

### 3.11 Escreva uma função para colocar num mesmo dataframe os dados de PIB percapita e índice de gini dos municípios brasieiros. 
#### Retorne os registros completos (PIB percaita, gini, UF, cod_ibge, Municipio) dos top 10 registros com maiores PIB percapita.
#### Analise a diferença entre os quartis (do PIB percapita e do índice de gini) dos top 10 municípios, em termos de PIB percapita,  e os quartis de todo o brasil para estas mesmas duas variáveis. Responda: um PIB percapita significativamente maior, tem ajudado a melhorar o índice de Gini nesses municípios ? Dica: use o método describe(). 
#### Esta análise não vale nota. Será avaliada a nota somente da função que retorna os top 10 registros.

In [59]:
# Dica 1: Use as funções criadas no cader de Exercícios desta aula.
# Dica 2: Ao ler o arquivo CSV do índice de gini, se preciso, use encoding='utf8' como parâmetro do método read_csv
def merge_pop_pib(path_pop, path_pib):
    """retorne um dataframe resultante do merge dos dados do pib e da população
    """
    # YOUR CODE HERE
    try:
        df_pib = pd.read_excel(path_pib, skiprows=3, skipfooter=1, index_col=0, dtype={'Unnamed: 1': str})
        df_pib.rename(columns={'Unnamed: 1' : 'cod_ibge7', 'Unnamed: 2' : 'municipio'}, inplace=True)        

        df_pop = pd.read_excel(path_pop, index_col=0, dtype={'cod_uf': str, 'cod_munic': str})
        df_pop['cod_ibge7'] = df_pop['cod_uf'] + df_pop['cod_munic']

        return pd.merge(df_pop, df_pib, on='cod_ibge7')
        
    except ValueError as e:
        raise NotImplementedError(e)

def pib_percapita(pib_pop):
    
    """retorne retorne um dataframe contendo o codigo ibge de 7 caracteres,
       o nome do municipio e seu PIB percapita.
    """
    # YOUR CODE HERE
    try:
        pib_pop['pib_percapita'] = pib_pop['2017'] / pib_pop['pop2017']
        return pib_pop
        
    except ValueError as e:
        
        raise NotImplementedError(e)


def top10(df, col):
    """retorne os top 10 registros do dataframe df ordenados por col
    """
    # YOUR CODE HERE
    try:
        df_topten = pd.DataFrame(df.sort_values(col, ascending=False).iloc[:10])

        # esse trecho foi necessario por conta do assert
        df_topten.rename(columns={'municipio_x' : 'municipio'}, inplace=True)
        list_col_assert = ['cod_ibge7', 'municipio', 'uf', 'pib_percapita', 'cod_ibge6', 'Município', '1991', '2000', '2010']

        return df_topten[list_col_assert]
    except:
        raise NotImplementedError('Algo deu errado.')

def top10_municipios(path_pib, path_gini):
    """retorne um dataframe contendo os 10 municipios com maiores PIB percapita. O dataframe deve conter também 
    o valor do índice de gini e nome, UF e código ibge do respectivo município.
    """
    # YOUR CODE HERE
    try:
        path_pop='../../curso_ciencia_dados2020/data/originais/populacao/estimativa_dou_2017.xlsx'
        df_pib_pop = merge_pop_pib(path_pop, path_pib)
        df_pib_percapita_2017 = pib_percapita(df_pib_pop)
        df_pib_percapita_2017['cod_ibge6'] = df_pib_percapita_2017['cod_ibge7'].apply(lambda x: x[:-1])
#         return df_pib_percapita_2017.head()
        df_gini = pd.read_csv(path_gini, sep=';', na_values='...', skiprows=2, skipfooter=1, decimal=',', \
                              dtype={'1991': float, '2000': float, '2010': float}, \
                               engine='python')

        df_gini['cod_ibge6'] = df_gini['Município'].apply( lambda x: x.split(' ')[0] )
        df_gini['Município'] = df_gini['Município'].apply( lambda x: ' '.join( x.split(' ')[1:] ) )
#         return df_gini
        df_full= pd.merge(df_pib_percapita_2017, df_gini, on='cod_ibge6', how='left')

       
        return top10(df_full, 'pib_percapita')
      
    except ValueError as e:
        raise NotImplementedError(e)



In [60]:
## Faça seus testes aqui
## Faça seus testes aqui
path_pib = '../../curso_ciencia_dados2020/data/originais/pib/pib_municipios.xlsx'
path_pop='../../curso_ciencia_dados2020/data/originais/populacao/estimativa_dou_2017.xlsx'
path_gini='../../curso_ciencia_dados2020/data/originais/gini/ginibr.csv'


# print(df_top10.describe())
# print(pib_gini.describe())
top10_municipios(path_pib, path_gini)

,cod_ibge7,municipio,uf,pib_percapita,cod_ibge6,Município,1991,2000,2010
3677,3536505,Paulínia,SP,344.847169,353650,Paulínia,0.4900,0.5734,0.4880
5068,4322004,Triunfo,RS,311.211920,432200,Triunfo,0.5525,0.5633,0.4441
3575,3527306,Louveira,SP,300.639389,352730,Louveira,0.4603,0.4802,0.5190
3155,3204302,Presidente Kennedy,ES,292.397121,320430,Presidente Kennedy,0.5542,0.6440,0.4958
2965,3161908,São Gonçalo do Rio Abaixo,MG,289.925462,316190,São Gonçalo do Rio Abaixo,0.5720,0.5335,0.4787
5174,5007802,Selvíria,MS,271.094724,500780,Selvíria,0.5981,0.5428,0.4551
2183,2929206,São Francisco do Conde,BA,253.895574,292920,São Francisco do Conde,0.6440,0.5873,0.5198
2523,3125101,Extrema,MG,219.239081,312510,Extrema,0.5166,0.4907,0.4015
293,1508357,Vitória do Xingu,PA,209.799918,150835,Vitória do Xingu,NaN,0.6751,0.6065
3547,3524709,Jaguariúna,SP,209.320862,352470,Jaguariúna,0.5785,0.5449,0.5020


In [61]:
# Validação para nota
test_data = {'cod_ibge7': {3677: '3536505', 5068: '4322004'},
 'municipio': {3677: 'Paulínia', 5068: 'Triunfo'},
 'uf': {3677: 'SP', 5068: 'RS'},
 'pib_percapita': {3677: 344.8471692406755, 5068: 311.2119198274948},
 'cod_ibge6': {3677: '353650', 5068: '432200'},
 'Município': {3677: 'Paulínia', 5068: 'Triunfo'},
 '1991': {3677: 0.49, 5068: 0.5525},
 '2000': {3677: 0.5734, 5068: 0.5633},
 '2010': {3677: 0.488, 5068: 0.4441}}
assert top10_municipios(path_pib, path_gini).iloc[:2].to_dict() == test_data